# Distributions, PDFs and Likelihoods

In this part we will introduce three tools that are the basics of statistics: distributions (discrete or continuous), probability distribution functions (PDFs) and likelihoods (+ negative log likelihoods).

Useful links:
- [Intro to Probability Distributions and Distribution Fitting with Python's SciPy](https://towardsdatascience.com/probability-distributions-with-pythons-scipy-3da89bf60565)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import rv_continuous, norm
from scipy.integrate import quad
from dataclasses import dataclass

In [3]:
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib widget

In [4]:
events = 10000

fig, ax = plt.subplots(1, 1)

@interact(mean=widgets.FloatSlider(min=0, max=20, step=1, value=10), std=widgets.FloatSlider(min=0, max=10, step=0.1, value=5))
def plot_distribution(mean, std):
    ax.clear()
    ax.set_xlim(-10, 30)
    ax.set_ylim(0, 0.2)
    data = norm.rvs(mean, std, size=events)
    ax.hist(data, bins=100, density=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=10.0, description='mean', max=20.0, step=1.0), FloatSlider(value=5.0, …

In [24]:
mean = 0
std = 5
min_x = -20
max_x = 20

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

@interact(left=widgets.FloatSlider(min=min_x, max=mean, step=0.1, value=min_x), right=widgets.FloatSlider(min=mean, max=max_x, step=0.1, value=max_x))
def update_integral(left, right):
    frozen = norm(loc=mean, scale=std)
    x = np.linspace(min_x, max_x, 10000)
    fx = frozen.pdf(x)
    Fx = frozen.cdf(x)
    res, err = quad(frozen.pdf, left, right)
    ax1.clear()
    ax2.clear()
    ax1.set_xlim(min_x, max_x)
    ax2.set_xlim(min_x, max_x)
    ax2.set_xlabel('x')
    ax1.set_ylabel('f(x)')
    ax2.set_ylabel('F(x)')
    ax1.plot(x, fx, 'k-')
    ptx = np.linspace(left, right, 100)
    ax1.fill_between(ptx, frozen.pdf(ptx), color='lightblue')
    ax1.text(0.2, 0.9, 'P[{:.2f}, {:.2f}] = {:.2f}'.format(left, right, res), horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
    ax2.plot(x, Fx, 'k-')
    ax2.hlines(frozen.cdf(left), min_x, left, color='r', linestyle='--')
    ax2.hlines(frozen.cdf(right), min_x, right, color='r', linestyle='--')
    ax2.vlines(left, 0, frozen.cdf(left), color='r', linestyle='--')
    ax2.vlines(right, 0, frozen.cdf(right), color='r', linestyle='--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-20.0, description='left', max=0.0, min=-20.0), FloatSlider(value=20.0…

In [22]:
@dataclass
class Likelihood:
    distribution: rv_continuous
    data: np.ndarray

    def __call__(self, *params):
        return np.prod(np.array([self.distribution.pdf(x, *params) for x in self.data]))

class NLL(Likelihood):
    def __call__(self, *params):
        return -np.sum(np.array([np.log(self.distribution.pdf(x, *params)) for x in self.data]))

In [39]:
mean = 10
std = 5
min_x = -10
max_x = 30

x = np.linspace(min_x, max_x, 1000) # sample x axis

fake_single_data = np.array([mean + 0.01 * mean])
norm_likelihood = Likelihood(norm, fake_single_data)

fig, ax = plt.subplots()

@interact(mean=widgets.FloatSlider(min=0, max=20, step=1, value=10), std=widgets.FloatSlider(min=4.5, max=8, step=0.1, value=5))
def plot_pdf(mean, std):
    lk_value = norm_likelihood(mean, std)
    ax.clear()
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(-0.01, 0.09)
    ax.plot(x, norm.pdf(x, mean, std), 'r-')
    ax.plot(fake_single_data, 0, 'o', color='#0b559f', markersize=10)
    ax.plot(fake_single_data, lk_value, 'o', color='#0b559f', markersize=10)
    ax.hlines(lk_value, min_x, fake_single_data[0], color='black', linestyle='dashed', alpha=0.5)
    ax.vlines(fake_single_data[0], 0, lk_value, color='black', linestyle='dashed', alpha=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('$L(\mu,\sigma | x)$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=10.0, description='mean', max=20.0, step=1.0), FloatSlider(value=5.0, …